This file accepts the CSV with randomized hyperparameter parameters then generates the directory for each hyperparameter experiment and populates it with a run script for the BlueWaters supercomputer, ROLLO input file, and OpenMC template file. 

In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os, json 
from jinja2 import Template
from shutil import copyfile

In [17]:
df = pd.read_csv("coarse_search_hyperparameters.csv", index_col="experiment_num")
experiments = len(df)

In [18]:
df

,pop_size,gens,mut_prob,mate_prob,select_op,k,tournsize,mut_op,mate_op
experiment_num,,,,,,,,,
0,21,29,0.157617,0.527051,selTournament,13,7.0,mutPolynomialBounded,cxOnePoint
1,63,10,0.166172,0.513330,selNSGA2,22,NaN,mutPolynomialBounded,cxBlend
2,93,7,0.296026,0.277389,selTournament,42,8.0,mutPolynomialBounded,cxBlend
3,67,9,0.177595,0.400426,selTournament,23,4.0,mutPolynomialBounded,cxBlend
4,87,7,0.276113,0.217605,selBest,40,NaN,mutPolynomialBounded,cxOnePoint
5,64,10,0.109872,0.372388,selTournament,21,3.0,mutPolynomialBounded,cxUniform
6,83,8,0.326200,0.175023,selTournament,38,7.0,mutPolynomialBounded,cxOnePoint
7,43,14,0.299562,0.446814,selBest,17,NaN,mutPolynomialBounded,cxBlend
8,86,7,0.308425,0.470644,selTournament,33,2.0,mutPolynomialBounded,cxOnePoint


In [19]:
experiments = 2
with open('bwscript_template', 'r') as f:
    bwscript = f.read() 
bwscript_template = Template(bwscript)

for i in range(35,40):
    # select operator dict 
    if df["select_op"][i] == "selTournament":
        select_op_dict = {"operator": df["select_op"][i], "k": int(df["k"][i]), "tournsize": int(df["tournsize"][i])}
    else: 
        select_op_dict = {"operator": df["select_op"][i], "k": int(df["k"][i])}
    # mutation operator dict
    if df["mate_op"][i] == "cxUniform":
        mate_op_dict = {"operator": df["mate_op"][i], "indpb": float(df["mate_prob"][i])}
    elif df["mate_op"][i] == "cxBlend":
        mate_op_dict = {"operator": df["mate_op"][i], "alpha": float(df["mate_prob"][i])}
    else:
        mate_op_dict = {"operator": df["mate_op"][i]}
    realm_input_dict = {
        "control_variables": {
            "sine_a": {"min": 0.0, "max": 2.0},
            "sine_b": {"min": 0.0, "max": 1.57},
            "sine_c": {"min": 0.0, "max": 6.28}
        },
        "evaluators": {
            "openmc": {
                "input_script": "openmcinp.py",
                "inputs": ["sine_a", "sine_b", "sine_c"],
                "outputs": ["keff"], 
                "keep_files": False
            }
        },
        "constraints": {"keff": {"operator": [">="], "constrained_val": [1.0]}},
        "algorithm": {
            "objective": "max",
            "optimized_variable": "keff",
            "pop_size": int(df["pop_size"][i]),
            "generations": int(df["gens"][i]),
            "mutation_probability": float(df["mut_prob"][i]),
            "mating_probability": float(df["mate_prob"][i]),
            "selection_operator": select_op_dict,
            "mutation_operator": {"operator":df["mut_op"][i], "eta": float(df["mut_prob"][i]), "indpb": float(df["mut_prob"][i])},
            "mating_operator": mate_op_dict
        },
    }
    folder_name = "coarse_search_"+str(i)
    os.mkdir(folder_name)
    copyfile("./openmcinp_template.py", folder_name+"/openmcinp.py")
    os.chdir(folder_name)
    with open('realm_input.json', 'w') as outfile:
        json.dump(realm_input_dict, outfile)
    wall_time = int(np.ceil(17/60*df["gens"][i]))
    bwscript_templated = bwscript_template.render(nodes=df["pop_size"][i]+1, 
                                                  walltime=str(wall_time)+":00:00", 
                                                  folder=folder_name)
    with open('run', 'w+') as fh:
        fh.write(bwscript_templated)
    os.chdir("../")